In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys

sys.path.append('./helpers_models/')
sys.path.append('./data_visualization_and_augmentations/')
sys.path.append('../torch_videovision/')
sys.path.append('./important_csvs/')

from helpers_resnet import *

resnet = torchvision.models.resnet50(pretrained=True)
adaptive_pooling = AdaptiveConcatPool2d()
head = Head()
resnet.avgpool = adaptive_pooling
resnet.fc = head

os.environ['CUDA_VISIBLE_DEVICES']='0,1,2'

resnet = resnet.cuda()

for param in resnet.parameters():
    param.requires_grad = False
    
for param in resnet.avgpool.parameters():
    param.requires_grad = True
    
for param in resnet.fc.parameters():
    param.requires_grad = True

resnet = nn.DataParallel(resnet)
check_freeze(resnet.module)

In [ ]:
from helpers_thresholds import *

In [ ]:
plt.rcParams['figure.figsize'] = (12,6)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 40}
plt.rc('font', **font)

In [ ]:
tensor_transform = get_tensor_transform('ImageNet', False)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

bs = 50
root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
df = pd.read_csv('./small_dataset_csvs/events_with_number_of_frames_stratified.csv')
# df_train = get_df(df, 20, True, False, False)
# class_image_paths, end_idx, idx_label= get_indices(df_train, root_dir)
# train_loader = get_loader(1, 128, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, True, True, 1)
df_valid = get_df(df, 20, False, True, False)
class_image_paths, end_idx, idx_label = get_indices(df_valid, root_dir)
valid_loader, valid_dataset = get_loader(1, bs, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, True, True, 1)
df_test = get_df(df, 20, False, False, True)
class_image_paths, end_idx, idx_label = get_indices(df_test, root_dir)
test_loader, test_dataset = get_loader(1, bs, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, True, True, 1)

torch.cuda.empty_cache()

In [ ]:
load = True
if load:
    checkpoint = torch.load('/media/scratch/astamoulakatos/saved-resnet-models/fifth-small-aug-final/best-checkpoint-010epoch.pth')
    resnet.load_state_dict(checkpoint['model_state_dict'])
    print('loading pretrained freezed model!')

In [ ]:
device = torch.device('cuda')

In [ ]:
from predictions import *

In [ ]:
resnet

In [ ]:
from skorch.net import NeuralNet

In [ ]:
net = NeuralNet(
    resnet.module,
    max_epochs=20,
    verbose=0,
    criterion=nn.BCEWithLogitsLoss(),
    device='cuda',
    iterator_train__shuffle=True,
    iterator_train__num_workers=4,
)

In [ ]:
net

In [ ]:
valid_dataset[0]

In [ ]:
net.fit(valid_dataset, y=None);

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance

In [ ]:
clf = BinaryRelevance(classifier=net, require_dense=[True,True])

In [ ]:
clf

In [ ]:
clf.fit(valid_loader.dataset)

In [ ]:
y_train

In [ ]:
labels = [y for x, y in iter(valid_loader.dataset)]

In [ ]:
labels

In [ ]:
torch.stack(labels)

In [ ]:
y_train = np.array(torch.stack(labels))

In [ ]:
x_train = valid_loader.dataset

In [ ]:
clf.fit(images, labels)

In [ ]:
len(y_train)

In [ ]:
len(x_train)

In [ ]:
len(valid_loader.dataset)

In [ ]:
labels = [y for x, y in iter(valid_loader.dataset)]

In [ ]:
images = [x for x, y in iter(valid_loader.dataset)]

In [ ]:
labels

In [ ]:
y_train = np.array(torch.cat(labels))

In [ ]:
len(y_train)

In [ ]:
images = 

In [ ]:
len(labels), len(images)

In [ ]:
torch.stack(labels)

In [ ]:
type(valid_loader.dataset)